# Comparação Dados Analógicos e Digitais

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import mlab
from scipy import signal
from scipy.fft import fftshift
from tqdm import tqdm
from scipy.signal import butter, lfilter
import seaborn as sns
import os
import math as m
import shutil
import random
import statistics as stc
from statistics import variance
import scipy.stats as stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Importando Dados Digitais Que Serão Comparados

In [ ]:
path_grav = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/Gravidade/'
filename = os.listdir( path_grav ) #Reading all file names inside the folder/path
filename #List of file names

In [ ]:
path_normal = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/22-04-2023-Normal/'
filenames = os.listdir( path_normal ) #Reading all file names inside the folder/path
filenames #List of file names

In [ ]:
path_desbal1_1 = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/10-05-2023-Desbalanceamento1/1_massa/'
filenames1_1 = os.listdir( path_desbal1_1 ) #Reading all file names inside the folder/path
filenames1_1 #List of file names

In [ ]:
#Getting gravity:
for names in filename:
    Gravity=pd.read_csv(path_grav + names, index_col=0,sep='\t')

#Other dataframes:
Digital_N={} #Dictionary of all datasets for normal conditions. The keys are the frequency and the informations are the dataframes for its corresponding frequency
for names in filenames:
    Digital_N[names.rstrip('.csv')]=pd.read_csv(path_normal + names, index_col=0,sep='\t') #The strip function is only for removing ".csv" from the key's name

Digital_D1_1={} #Dictionary of all datasets for unbalance conditions - type 1, mass 1. The keys are the frequency and the informations are the dataframes for its corresponding frequency
for names in filenames1_1:
    Digital_D1_1[names.rstrip('.csv')]=pd.read_csv(path_desbal1_1 + names, index_col=0,sep='\t') #The strip function is only for removing ".csv" from the key's name


In [ ]:
#Digital Informations:
Dictionaries_names=['Digital_N','Digital_D1_1'] #IMPORTANT INFORMATION (MANUALLY INFORM AFTER ADDING OTHER DICTIONARY OF DATAFRAMES)
a_X=['imu1accx','imu2accx','imu3accx'] # information of X-acceleration
a_Y=['imu1accy','imu2accy','imu3accy'] # information of Y-acceleration
a_Z=['imu1accz','imu2accz','imu3accz'] # information of Z-acceleration
g_X=['imu1gyrox','imu2gyrox','imu3gyrox'] # information of X-gyroscope
g_Y=['imu1gyroy','imu2gyroy','imu3gyroy'] # information of Y-gyroscope
g_Z=['imu1gyroz','imu2gyroz','imu3gyroz'] # information of Z-gyroscope
M_1_a=['imu1accx','imu1accy','imu1accz']
M_2_a=['imu2accx','imu2accy','imu2accz']
M_3_a=['imu3accx','imu3accy','imu3accz']
M_1_g=['imu1gyrox','imu1gyroy','imu1gyroz']
M_2_g=['imu2gyrox','imu2gyroy','imu2gyroz']
M_3_g=['imu3gyrox','imu3gyroy','imu3gyroz']

In [ ]:
#Deleting garbage information
list_remov=['dac1','dac2','dac3','dac4','log']

def delgarbage (Dict_of_df): #Give the Name of the Dictionary with Dataframes
    for keys in Dict_of_df:
      for col_to_remove in list_remov:
        Dict_of_df[keys].drop(col_to_remove, axis=1, inplace=True)
    remove=list_remov
    informations=list(Dict_of_df[list(Dict_of_df)[0]].columns)
    return remove, informations


#Removing Gravity from the X-axis
def removegravity (Dict_of_df):
    for keys in Dict_of_df:
        for info in a_X:
            Dict_of_df[keys][info] -= Gravity[info].mean()
    return

In [ ]:
#Loop to remove garbage information / columns
for n in Dictionaries_names:
    l_removed, l_informations = delgarbage(eval(n))
print(" List of Removed Informations: ",l_removed,"\n","List of Useful Informations: ", l_informations)


In [ ]:
for n in Dictionaries_names:
    removegravity(eval(n))

### Tacometro para Sensores Digitais

In [ ]:
def Tachometer (Dict_of_df):

  for keys in Dict_of_df:
    df=Dict_of_df[keys][['time','imu1accx','imu1accy','imu1accz','imu2accx','imu2accy','imu2accz','imu3accx','imu3accy','imu3accz','adc1.3','adc2.3','adc3.3','adc4.3']]
    df_array=df.values
    tach=df[['adc1.3','adc2.3','adc3.3','adc4.3']].values.reshape((df.shape[0]*4))#Tranformando todas as informações do tacometro em 1 coluna só
    #print(tach)
    tach_size=tach.shape[0]
    i=0
    new_rot=0
    for i in range(1000,tach_size//2,500):
      it=0
      rot=new_rot
      interm=tach[i-500:i]
      interm_size=interm.shape[0]
      sig1 = (interm > 0.1).astype(int)
      sig2 = np.diff(sig1)
      idxpicos = np.where(sig2 > 0.5)[0]
      # A diferença temporal entre picos indica o perído de rotação:
      periodos = (idxpicos[1:] - idxpicos[0:-1]) * 1e-3  # Amostragem de 1 ms, por isso o 1e-3
      # Frequencias em RPM:
      new_rot = 1 / (np.mean(periodos)) * 60
      if abs(new_rot-rot)<1:
        if it<2:
          it+=1
        else:
          final_rot=new_rot
          break
      elif new_rot<rot:
        final_rot=rot
        break
      i+=1
    t_sub=df['time'][(i-it)//4]
    sig1 = (tach[i-500:i+9500] > 0.1).astype(int)
    sig2 = np.diff(sig1)
    idxpicos = np.where(sig2 > 0.5)[0]
    periodos = (idxpicos[1:] - idxpicos[0:-1]) * 1e-3
    rot = 1 / (np.mean(periodos)) * 60
    print(f'Para {keys} o tempo de subida e rotação foram:')
    print(f'O tempo de subida é {t_sub} s')
    print('Ponto/Linha para Primeiro Corte:', t_sub/0.004)
    print('Ponto/Linha para Segundo Corte:', (t_sub/0.004)+2500) #10s/0.004=2500
    print(f'A velocidade máxima de rotação é {rot} rpm')
    print()
  return t_sub , rot

In [ ]:
def Tachometer_for_funct (Dict_of_df, keys):
  df=Dict_of_df[keys][['time','imu1accx','imu1accy','imu1accz','imu2accx','imu2accy','imu2accz','imu3accx','imu3accy','imu3accz','adc1.3','adc2.3','adc3.3','adc4.3']]
  df_array=df.values
  tach=df[['adc1.3','adc2.3','adc3.3','adc4.3']].values.reshape((df.shape[0]*4))#Tranformando todas as informações do tacometro em 1 coluna só
  #print(tach)
  tach_size=tach.shape[0]
  i=0
  new_rot=0
  for i in range(1000,tach_size//2,500):
    it=0
    rot=new_rot
    interm=tach[i-500:i]
    interm_size=interm.shape[0]
    sig1 = (interm > 0.1).astype(int)
    sig2 = np.diff(sig1)
    idxpicos = np.where(sig2 > 0.5)[0]
    # A diferença temporal entre picos indica o perído de rotação:
    periodos = (idxpicos[1:] - idxpicos[0:-1]) * 1e-3  # Amostragem de 1 ms, por isso o 1e-3
    # Frequencias em RPM:
    new_rot = 1 / (np.mean(periodos)) * 60
    if abs(new_rot-rot)<1:
      if it<2:
        it+=1
      else:
        final_rot=new_rot
        break
    elif new_rot<rot:
      final_rot=rot
      break
    i+=1
  t_sub=(df['time'][(i-it)//4])   #Pode-se colocar um adcional para garantir a faixa constante
  sig1 = (tach[i-500:i+9500] > 0.1).astype(int)
  sig2 = np.diff(sig1)
  idxpicos = np.where(sig2 > 0.5)[0]
  periodos = (idxpicos[1:] - idxpicos[0:-1]) * 1e-3
  rot = 1 / (np.mean(periodos)) * 60
  stop1=int(t_sub/0.004)
  stop2=int((t_sub/0.004)+2500) #10s/0.004=2500
  #print(f'Para {keys} o tempo de subida e rotação foram:')
  #print(f'O tempo de subida é {t_sub} s')
  #print('Ponto/Linha para Primeiro Corte:', stop1)
  #print('Ponto/Linha para Segundo Corte:', stop2)
  #print(f'A velocidade máxima de rotação é {rot} rpm')
  #print()
  return stop1 , stop2, rot

## Importanto Dados Analógicos que Serão Comparados

In [ ]:
path_ana1 = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/26-04-2023-AnalogicoNormal/Normal/'
filename_ana1 = os.listdir( path_ana1 ) #Reading all file names inside the folder/path
filename_ana1 #List of file names

In [ ]:
path_ana2 = '/content/drive/My Drive/IC/ColabNotebooks/Vibracoes/DataFrames/03-05-2023-AnalogicoDesbalanceamento1/'
filename_ana2 = os.listdir( path_ana2 ) #Reading all file names inside the folder/path
filename_ana2 #List of file names

In [ ]:
# Convertion Table (Digital - Analogic):

# X (analógico) = Z (digital)
# Y (analógico) = - Y (digital)
# Z (analógico) = X (digital)

# We set all information as digital, therefore we changed the columns names for the corresponding digital name

#Other dataframes:
Analog_N={} #Dictionary of all datasets for normal conditions. The keys are the frequency and the informations are the dataframes for its corresponding frequency
for names in filename_ana1:
    Analog_N[names[names.find('-')+1:names.find("q")]]=pd.read_csv(path_ana1 + names,skiprows=2, index_col=False,sep='\t',decimal=',',names=['Tempo', 'Eixo_z', 'Eixo_-y', 'Eixo_x','Sinal_4'],dtype = {'Tempo': float, 'Eixo_z': float, 'Eixo_-y': float,'Eixo_x': float,'Sinal_4':float}) #The strip function is only for removing ".csv" from the key's name

Analog_D1_1={} #Dictionary of all datasets for unbalance conditions - type 1, mass 1. The keys are the frequency and the informations are the dataframes for its corresponding frequency
for names in filename_ana2:
    Analog_D1_1[names[names.find('-')+1:names.find("q")]]=pd.read_csv(path_ana2 + names,skiprows=2, index_col=False,sep='\t',decimal=',',names=['Tempo', 'Eixo_z', 'Eixo_-y', 'Eixo_x','Sinal_4'],dtype = {'Tempo': float, 'Eixo_z': float, 'Eixo_-y': float,'Eixo_x': float,'Sinal_4':float}) #The strip function is only for removing ".csv" from the key's name


In [ ]:
Dictionaries_names_A=['Analog_N','Analog_D1_1'] #IMPORTANT INFORMATION (MANUALLY INFORM AFTER ADDING OTHER DICTIONARY OF DATAFRAMES)
A_acc= ['Eixo_z', 'Eixo_-y', 'Eixo_x']

In [ ]:
for keys in Analog_N:
  print(Analog_N[keys])
  print(keys)

In [ ]:
for keys in Analog_D1_1:
  print(Analog_D1_1[keys])

In [ ]:
list_remov_A=['Sinal_4']

def delgarbage_A (Dict_of_df): #Give the Name of the Dictionary with Dataframes
    for keys in Dict_of_df:
      for col_to_remove in list_remov_A:
        Dict_of_df[keys].drop(col_to_remove, axis=1, inplace=True)
    remove_A=list_remov_A
    informations_A=list(Dict_of_df[list(Dict_of_df)[0]].columns)
    return remove_A, informations_A

In [ ]:
#Loop to remove garbage information / columns
for n in Dictionaries_names_A:
    l_removed_A, l_informations_A = delgarbage_A(eval(n))
print(" List of Removed Informations: ",l_removed_A,"\n","List of Useful Informations: ", l_informations_A)


## Espectogramas Dados Digitais

In [ ]:
plt.rcParams['figure.max_open_warning'] = 100

In [ ]:
plt.rcParams.update({'font.size': 15})

In [ ]:
#Printing Spectograms (Only Magnitude):
#THE FUNCTION fspectrum ONLY ACCEPT A data list OF 3 VALUES OR MULTIPLES OF 3
def pSpectogram (Dict_of_df, data=a_X): #Data uses always the X axis ---> DATA ONLY ACCEPT 3 COMPONENTS
    if len(data)!=3:
      print("Data doesn't have three features")

    else:
      print('Escalas: 1 - Linear ou 2 - dB')
      flag=int(input("Qual escala você gostaria de usar ?:"))

      for keys in Dict_of_df: #Accessing the dictionary with the first keys (RPM,sample1,...)

        stop1 , stop2, rot = Tachometer_for_funct (Dict_of_df, keys)

        #Each key have its maximum time and number of points collected, therefore, the initial calculations for FFT are made:
        # Now we are going to get the period of each dataframe / rotational frequency and create a list of spectral frequencies to analyze the frequency domain
        # It's notable that we are working with uniform sampling, therefore our sampling frequency will be constant

        T=Dict_of_df[keys]['time'].max()
        N=len(Dict_of_df[keys]['time'])
        delta_t=T/N
        fs=1/delta_t #Sampling frequency (Hz)
        #print(fs) #Vai dar diferente de 250Hz, pois cartamos o sinal um pouco antes ou depois de fechar o ciclo completo de captura

        fig, axs = plt.subplots(1,3,figsize=(20, 10))
        fig.suptitle(keys +'-'+ str(round(rot,2))+'RPM / '+str(round((rot/60),2))+'Hz '+ '- Spectrograms - Digital Accelerometer', fontsize=30, fontweight='bold')

        f1, t1, Sxx1 = signal.spectrogram(Dict_of_df[keys][data[0]], fs=fs,window=('hann'),nperseg=250,mode='magnitude',noverlap=125)
        f2, t2, Sxx2 = signal.spectrogram(Dict_of_df[keys][data[1]], fs=fs,window=('hann'),nperseg=250,mode='magnitude',noverlap=125)
        f3, t3, Sxx3 = signal.spectrogram(Dict_of_df[keys][data[2]], fs=fs,window=('hann'),nperseg=250,mode='magnitude',noverlap=125)

        if flag==1:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          #print('max=',np.max(Sxx1[:,0:-15]),np.max(Sxx2[:,0:-15]),np.max(Sxx3[:,0:-15]))

          #Printing Graphs
          axs[0].pcolormesh(t1[0:-15], f1, Sxx1[:,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[0].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2[0:-15], f2, Sxx2[:,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t3[0:-15], f3, Sxx3[:,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[2].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)

          cbar=fig.colorbar(plt.pcolormesh(t3[0:-15], f3, Sxx3[:,0:-15], shading='gouraud',vmin=0,vmax=0.03),label='m/s² (Linear Scale)')
          plt.tight_layout()

        elif flag==2:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          Sxx1=20*np.log10(Sxx1)
          Sxx2=20*np.log10(Sxx2)
          Sxx3=20*np.log10(Sxx3)

          #print('max=',np.max(Sxx1[:,0:-15]),np.max(Sxx2[:,0:-15]),np.max(Sxx3[:,0:-15]))
          #print('min=',np.min(Sxx1[:,0:-15]),np.min(Sxx2[:,0:-15]),np.min(Sxx3[:,0:-15]))

          #Printing Graphs
          axs[0].pcolormesh(t1[0:-15], f1, Sxx1[:,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[0].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2[0:-15], f2, Sxx2[:,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t3[0:-15], f3, Sxx3[:,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[2].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)
          cbar=fig.colorbar(plt.pcolormesh(t3[0:-15], f3, Sxx3[:,0:-15], shading='gouraud',vmin=-80,vmax=0),label='m/s² (dB Scale)')
          plt.tight_layout()

    return

#https://www.mathworks.com/help/matlab/ref/clim.html;jsessionid=17d8c63e8b64966098b7dfa9a14b
#https://www.mathworks.com/matlabcentral/answers/881898-how-can-i-change-the-colorbar-limits-scale-of-a-spectrogram
#https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html
#https://proplot.readthedocs.io/en/stable/colorbars_legends.html


In [ ]:
#TESTE
fig = plt.plot(figsize=(40, 100))
plt.title('Colormesh', fontsize=30, fontweight='bold')
f1, t1, Sxx1 = signal.spectrogram(Digital_D1_1['sample8']['imu3accx'], fs=250,window=('hann'),nperseg=250,mode='magnitude',noverlap=125)
print(20*np.log10(Sxx1))
print(np.max(20*np.log10(Sxx1)))
print(np.min(20*np.log10(Sxx1)))
plt.pcolormesh(t1[0:-15], f1, 20*np.log10(Sxx1)[:,0:-15], shading='gouraud',vmin=-80,vmax=0)
plt.title(" Spectrogram ",fontsize=20)
plt.xlabel('Time (s)',fontsize=20)
plt.ylabel('Frequency (Hz)',fontsize=20)
plt.colorbar()

### Normal

#### Mancal 1

In [ ]:
pSpectogram(Digital_N,data=M_1_a)

In [ ]:
pSpectogram(Digital_N,data=M_1_a)

#### Mancal 2

In [ ]:
pSpectogram(Digital_N,data=M_2_a)

In [ ]:
pSpectogram(Digital_N,data=M_2_a)

#### Mancal 3

In [ ]:
pSpectogram(Digital_N,data=M_3_a)

In [ ]:
pSpectogram(Digital_N,data=M_3_a)

### Desbalaceamento

#### Mancal 1

In [ ]:
pSpectogram(Digital_D1_1, data=M_1_a)

In [ ]:
pSpectogram(Digital_D1_1, data=M_1_a)

#### Mancal 2

In [ ]:
pSpectogram(Digital_D1_1, data=M_2_a)

In [ ]:
pSpectogram(Digital_D1_1, data=M_2_a)

#### Mancal 3

In [ ]:
pSpectogram(Digital_D1_1, data=M_3_a)

In [ ]:
pSpectogram(Digital_D1_1, data=M_3_a)

## Espectograma Dados Analógicos

### Espectrogramas Completos

In [ ]:
#Printing Spectograms (Only Magnitude):
#THE FUNCTION fspectrum ONLY ACCEPT A data list OF 3 VALUES OR MULTIPLES OF 3
def pSpectogram_A_Full (Dict_of_df, data=A_acc): #Data uses always the X axis ---> DATA ONLY ACCEPT 3 COMPONENTS
    if len(data)!=3:
      print("Data doesn't have three features")

    else:
      print('Escalas: 1 - Linear ou 2 - dB')
      flag=int(input("Qual escala você gostaria de usar ?:"))
      for keys in Dict_of_df: #Accessing the dictionary with the first keys (RPM,sample1,...)

        #Each key have its maximum time and number of points collected, therefore, the initial calculations for FFT are made:
        # Now we are going to get the period of each dataframe / rotational frequency and create a list of spectral frequencies to analyze the frequency domain
        # It's notable that we are working with uniform sampling, therefore our sampling frequency will be constant

        T=Dict_of_df[keys]['Tempo'].max()
        N=len(Dict_of_df[keys]['Tempo'])
        delta_t=T/N
        fs=1/delta_t #Sampling frequency (Hz)
        #print(fs) #Tem que da 1000Hz

        fig, axs = plt.subplots(1,3,figsize=(20, 10))
        fig.suptitle(keys + ' - Spectrograms - Analog Accelerometer', fontsize=30, fontweight='bold')

        f1, t1, Sxx1 = signal.spectrogram(Dict_of_df[keys][data[0]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)
        f2, t2, Sxx2 = signal.spectrogram(Dict_of_df[keys][data[1]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)
        f3, t3, Sxx3 = signal.spectrogram(Dict_of_df[keys][data[2]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)

        if flag==1:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          print('max=',np.max(Sxx1[:,:]),np.max(Sxx2[:,:]),np.max(Sxx3[:,:]))

          #Printing Graphs
          axs[0].pcolormesh(t1, f1, Sxx1, shading='gouraud',vmin=0,vmax=0.03)
          axs[0].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2, f2, Sxx2, shading='gouraud',vmin=0,vmax=0.03)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t3, f3, Sxx3, shading='gouraud',vmin=0,vmax=0.03)
          axs[2].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)

          cbar=fig.colorbar(plt.pcolormesh(t3, f3, Sxx3, shading='gouraud',vmin=0,vmax=0.03),label='m/s² (Linear Scale)')
          plt.tight_layout()

        elif flag==2:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          Sxx1=20*np.log10(Sxx1)
          Sxx2=20*np.log10(Sxx2)
          Sxx3=20*np.log10(Sxx3)

          print('max=',np.max(Sxx1[:,:]),np.max(Sxx2[:,:]),np.max(Sxx3[:,:]))
          print('min=',np.min(Sxx1[:,:]),np.min(Sxx2[:,:]),np.min(Sxx3[:,:]))

          #Printing Graphs
          axs[0].pcolormesh(t1, f1, Sxx1, shading='gouraud',vmin=-80,vmax=0)
          axs[0].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2, f2, Sxx2, shading='gouraud',vmin=-80,vmax=0)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t3, f3, Sxx3, shading='gouraud',vmin=-80,vmax=0)
          axs[2].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)
          cbar=fig.colorbar(plt.pcolormesh(t3, f3, Sxx3, shading='gouraud',vmin=-80,vmax=0),label='m/s² (dB Scale)')
          plt.tight_layout()

    return

#https://www.mathworks.com/help/matlab/ref/clim.html;jsessionid=17d8c63e8b64966098b7dfa9a14b
#https://www.mathworks.com/matlabcentral/answers/881898-how-can-i-change-the-colorbar-limits-scale-of-a-spectrogram
#https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html
#https://proplot.readthedocs.io/en/stable/colorbars_legends.html


In [ ]:
pSpectogram_A_Full(Analog_N)

In [ ]:
pSpectogram_A_Full(Analog_D1_1)

### Espectrogramas Cortados

In [ ]:
#Printing Spectograms (Only Magnitude):
#THE FUNCTION fspectrum ONLY ACCEPT A data list OF 3 VALUES OR MULTIPLES OF 3
def pSpectogram_A (Dict_of_df, data=A_acc): #Data uses always the X axis ---> DATA ONLY ACCEPT 3 COMPONENTS
    if len(data)!=3:
      print("Data doesn't have three features")

    else:
      print('Escalas: 1 - Linear ou 2 - dB')
      flag=int(input("Qual escala você gostaria de usar ?:"))

      for keys in Dict_of_df: #Accessing the dictionary with the first keys (RPM,sample1,...)

        #Each key have its maximum time and number of points collected, therefore, the initial calculations for FFT are made:
        # Now we are going to get the period of each dataframe / rotational frequency and create a list of spectral frequencies to analyze the frequency domain
        # It's notable that we are working with uniform sampling, therefore our sampling frequency will be constant

        T=Dict_of_df[keys]['Tempo'].max()
        N=len(Dict_of_df[keys]['Tempo'])
        delta_t=T/N
        fs=1/delta_t #Sampling frequency (Hz)
        #print(fs) #Vai dar diferente de 250Hz, pois cartamos o sinal um pouco antes ou depois de fechar o ciclo completo de captura

        fig, axs = plt.subplots(1,3,figsize=(20, 10))
        fig.suptitle(keys + ' RPM - Spectrograms - Analog Accelerometer', fontsize=30, fontweight='bold')

        f1, t1, Sxx1 = signal.spectrogram(Dict_of_df[keys][data[0]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)
        f2, t2, Sxx2 = signal.spectrogram(Dict_of_df[keys][data[1]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)
        f3, t3, Sxx3 = signal.spectrogram(Dict_of_df[keys][data[2]], fs=fs,window=('hann'),nperseg=1000,mode='magnitude',noverlap=500)

        if flag==1:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          #print('max=',np.max(Sxx1[:125,0:-15]),np.max(Sxx2[:125,0:-15]),np.max(Sxx3[:125,0:-15]))

          #Printing Graphs
          axs[0].pcolormesh(t3[0:-15], f3[:125], Sxx3[:125,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[0].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2[0:-15], f2[:125], Sxx2[:125,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t1[0:-15], f1[:125], Sxx1[:125,0:-15], shading='gouraud',vmin=0,vmax=0.03)
          axs[2].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)

          cbar=fig.colorbar(plt.pcolormesh(t3[0:-15], f3[:125], Sxx3[:125,0:-15], shading='gouraud',vmin=0,vmax=0.03),label='m/s² (Linear Scale)')
          plt.tight_layout()

        elif flag==2:
          #print(f1)
          #print(t1)
          #print(Sxx1)
          Sxx1=20*np.log10(Sxx1)
          Sxx2=20*np.log10(Sxx2)
          Sxx3=20*np.log10(Sxx3)

          #print('max=',np.max(Sxx1[:125,0:-15]),np.max(Sxx2[:125,0:-15]),np.max(Sxx3[:125,0:-15]))
          #print('min=',np.min(Sxx1[:125,0:-15]),np.min(Sxx2[:125,0:-15]),np.min(Sxx3[:125,0:-15]))

          #Printing Graphs
          axs[0].pcolormesh(t3[0:-15], f3[:125], Sxx3[:125,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[0].set_title(keys+" --- "+data[2]+ " Spectrogram ",fontsize=20)
          axs[0].set_xlabel('Time (s)',fontsize=20)
          axs[0].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[1].pcolormesh(t2[0:-15], f2[:125], Sxx2[:125,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[1].set_title(keys+" --- "+data[1]+ " Spectrogram ",fontsize=20)
          axs[1].set_xlabel('Time (s)',fontsize=20)
          axs[1].set_ylabel('Frequency (Hz)',fontsize=20)

          axs[2].pcolormesh(t1[0:-15], f1[:125], Sxx1[:125,0:-15], shading='gouraud',vmin=-80,vmax=0)
          axs[2].set_title(keys +" --- "+data[0]+ " Spectrogram ",fontsize=20)
          axs[2].set_xlabel('Time (s)',fontsize=20)
          axs[2].set_ylabel('Frequency (Hz)',fontsize=20)

          cbar=fig.colorbar(plt.pcolormesh(t3[0:-15], f3[:125], Sxx3[:125,0:-15], shading='gouraud',vmin=-80,vmax=0),label='m/s² (dB Scale)')
          plt.tight_layout()

    return

#https://www.mathworks.com/help/matlab/ref/clim.html;jsessionid=17d8c63e8b64966098b7dfa9a14b
#https://www.mathworks.com/matlabcentral/answers/881898-how-can-i-change-the-colorbar-limits-scale-of-a-spectrogram
#https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html
#https://proplot.readthedocs.io/en/stable/colorbars_legends.html


#### Normal

In [ ]:
pSpectogram_A(Analog_N)

In [ ]:
pSpectogram_A(Analog_N)

#### Desbalanceado

In [ ]:
pSpectogram_A(Analog_D1_1)

In [ ]:
pSpectogram_A(Analog_D1_1)